In [1]:
from ipywidgets import widgets, interact
from __future__ import print_function
import plotly.plotly as py
import plotly.graph_objs as go
import cufflinks as cf
import pandas as pd
import numpy as np
import re

In [2]:
from IPython.display import display

In [4]:
raw_df = pd.read_csv('bevgeburtenjahrgeschlquartstz.csv')
columns = [f for f in raw_df]
columns[0] = "Jahr"
raw_df.columns = columns
raw_df[:3]

Jahr  SexCd SexKurz  StatZoneSort       QuarLang  QuarSort QuarLang  \
0  2015      2       W          1101   Zähringerstrasse        11  Rathaus   
1  2015      1       M          1101   Zähringerstrasse        11  Rathaus   
2  2015      2       W          1102  Zentralbibliothek        11  Rathaus   

   AnzGebuWir  
0           2  
1           2  
2           1

In [10]:
YEAR_MIN = min(raw_df['Jahr'])
YEAR_MAX = max(raw_df['Jahr'])

In [5]:
def get_selected(df, year_from, year_to, area):
    selected_df = df[(raw_df['Jahr']>=year_from) & (df['Jahr']<=year_to) & (df['QuarLang']==area)] # todo: explain it includes borders
    return selected_df

def get_structured(df):
    df = df.groupby(['SexKurz'])

    male_data = df.get_group('M').groupby(['Jahr']).apply(lambda x: x['AnzGebuWir'].sum()).reset_index()
    male_data.columns = ['Jahr', 'Male']

    female_data = df.get_group('W').groupby(['Jahr']).apply(lambda x: x['AnzGebuWir'].sum()).reset_index()
    female_data.columns = ['Jahr', 'Female']

    combined_data = male_data.merge(female_data)
    result = combined_data.set_index(combined_data.Jahr.values).drop('Jahr',1)
    return result

def years_valid(yfrom, yto):
    try:
        yfrom = int(yfrom)
        yto = int(yto)
        if yfrom>=YEAR_MIN and yto<=YEAR_MAX and yto>=yfrom:
            return True
        return False
    except:
        return False

In [55]:

def visualise3():
    areas = tuple(sorted(set(raw_df.QuarLang.values)))

    w_means = widgets.Text(
        value="",
        description='Mean Male / Female:',
        disabled=True
    )
    w_std = widgets.Text(
        value="",
        description='Std Male / Female:',
        disabled=True
    )

    @interact(
        year_from=widgets.Text(value='1999', description="Year from:"), 
        year_to=widgets.Text(value='2015', description="Year to:"),
        area=widgets.Dropdown(options=areas, description="Area:"),
        __manual=False)
    def update(year_from, year_to, area):
        if years_valid(year_from, year_to):
            year_from = int(year_from)
            year_to = int(year_to)

            trace0 = go.Bar(
                x = [2013,2014,2015]
            )

            result = get_structured(get_selected(raw_df, year_from, year_to, area))

            cf.set_config_file(offline=True, world_readable=True, theme='ggplot')
            result.iplot(
                kind='bar', 
                barmode='stack', 
                filename='cufflinks/stacked-bar-chart', 
                title='Births per Year in {}'.format(area),
                xTitle='Year',
                yTitle='Number of births')

            means = np.mean(result)
            stds = np.std(result)
            w_means.value = "{:.3f} / {:.3f}".format(means.Male, means.Female)
            w_std.value = "{:.3f} / {:.3f}".format(stds.Male, stds.Female)

        else:
            print('Provided values are invalid.')
            print('Years are allowed to range from {} to {}.'.format(YEAR_MIN, YEAR_MAX))
            w_means.value = ""
            w_std.value = ""
            return


    display(w_means)
    display(w_std)

In [56]:
visualise3()